In [ ]:
from __future__ import print_function

import sys
import numpy as np
from time import time
import matplotlib.pyplot as plt 

sys.path.append('/home/xilinx')
from pynq import Overlay
from pynq import allocate
from pynq import MMIO

ROM_SIZE = 0x2000 #8K

SOC_UP = 0x0000;
SOC_LA = 0x1000;
PL_AA_MB = 0x2000;
PL_AA = 0x2100;
SOC_IS = 0x3000;
SOC_AS = 0x4000;
SOC_CC = 0x5000;
PL_AS = 0x6000;
PL_IS = 0x7000;
PL_DMA = 0x8000;

In [ ]:
ol = Overlay("/home/xilinx/jupyter_notebooks/PS/caravel_fpga.bit")
#ol.ip_dict

In [ ]:
ipOUTPIN = ol.output_pin_0
ipPS = ol.caravel_ps_0
ipReadROMCODE = ol.read_romcode_0

In [ ]:
# Create np with 8K/4 (4 bytes per index) size and be initiled to 0
rom_size_final = 0

npROM = np.zeros(ROM_SIZE >> 2, dtype=np.uint32)
npROM_index = 0
npROM_offset = 0
fiROM = open("/home/xilinx/jupyter_notebooks/PS/fsic.hex", "r+")
#fiROM = open("counter_wb.hex", "r+")

for line in fiROM:
    # offset header
    if line.startswith('@'):
        # Ignore first char @
        npROM_offset = int(line[1:].strip(b'\x00'.decode()), base = 16)
        npROM_offset = npROM_offset >> 2 # 4byte per offset
        #print (npROM_offset)
        npROM_index = 0
        continue
    #print (line)

    # We suppose the data must be 32bit alignment
    buffer = 0
    bytecount = 0
    for line_byte in line.strip(b'\x00'.decode()).split():
        buffer += int(line_byte, base = 16) << (8 * bytecount)
        bytecount += 1
        # Collect 4 bytes, write to npROM
        if(bytecount == 4):
            npROM[npROM_offset + npROM_index] = buffer
            # Clear buffer and bytecount
            buffer = 0
            bytecount = 0
            npROM_index += 1
            #print (npROM_index)
            continue
    # Fill rest data if not alignment 4 bytes
    if (bytecount != 0):
        npROM[npROM_offset + npROM_index] = buffer
        npROM_index += 1
    
fiROM.close()

rom_size_final = npROM_offset + npROM_index
#print (rom_size_final)

#for data in npROM:
#    print (hex(data))


In [ ]:
# Allocate dram buffer will assign physical address to ip ipReadROMCODE

#rom_buffer = allocate(shape=(ROM_SIZE >> 2,), dtype=np.uint32)
rom_buffer = allocate(shape=(rom_size_final,), dtype=np.uint32)

# Initial it by npROM
#for index in range (ROM_SIZE >> 2):
for index in range (rom_size_final):
    rom_buffer[index] = npROM[index]
    
#for index in range (ROM_SIZE >> 2):
#    print ("0x{0:08x}".format(rom_buffer[index]))

# Program physical address for the romcode base address


# 0x00 : Control signals
#        bit 0  - ap_start (Read/Write/COH)
#        bit 1  - ap_done (Read/COR)
#        bit 2  - ap_idle (Read)
#        bit 3  - ap_ready (Read)
#        bit 7  - auto_restart (Read/Write)
#        others - reserved
# 0x10 : Data signal of romcode
#        bit 31~0 - romcode[31:0] (Read/Write)
# 0x14 : Data signal of romcode
#        bit 31~0 - romcode[63:32] (Read/Write)
# 0x1c : Data signal of length_r
#        bit 31~0 - length_r[31:0] (Read/Write)

ipReadROMCODE.write(0x10, rom_buffer.device_address)
ipReadROMCODE.write(0x1C, rom_size_final)

ipReadROMCODE.write(0x14, 0)

# ipReadROMCODE start to move the data from rom_buffer to bram
ipReadROMCODE.write(0x00, 1) # IP Start
while (ipReadROMCODE.read(0x00) & 0x04) == 0x00: # wait for done
    continue
    
print("Write to bram done")


In [ ]:
# Check MPRJ_IO input/out/en
# 0x10 : Data signal of ps_mprj_in
#        bit 31~0 - ps_mprj_in[31:0] (Read/Write)
# 0x14 : Data signal of ps_mprj_in
#        bit 5~0 - ps_mprj_in[37:32] (Read/Write)
#        others  - reserved
# 0x1c : Data signal of ps_mprj_out
#        bit 31~0 - ps_mprj_out[31:0] (Read)
# 0x20 : Data signal of ps_mprj_out
#        bit 5~0 - ps_mprj_out[37:32] (Read)
#        others  - reserved
# 0x34 : Data signal of ps_mprj_en
#        bit 31~0 - ps_mprj_en[31:0] (Read)
# 0x38 : Data signal of ps_mprj_en
#        bit 5~0 - ps_mprj_en[37:32] (Read)
#        others  - reserved

print ("0x10 = ", hex(ipPS.read(0x10)))
print ("0x14 = ", hex(ipPS.read(0x14)))
print ("0x1c = ", hex(ipPS.read(0x1c)))
print ("0x20 = ", hex(ipPS.read(0x20)))
print ("0x34 = ", hex(ipPS.read(0x34)))
print ("0x38 = ", hex(ipPS.read(0x38)))


In [ ]:
# Release Caravel reset
# 0x10 : Data signal of outpin_ctrl
#        bit 0  - outpin_ctrl[0] (Read/Write)
#        others - reserved
print (ipOUTPIN.read(0x10))
ipOUTPIN.write(0x10, 1)
print (ipOUTPIN.read(0x10))

In [ ]:
# Check MPRJ_IO input/out/en
# 0x10 : Data signal of ps_mprj_in
#        bit 31~0 - ps_mprj_in[31:0] (Read/Write)
# 0x14 : Data signal of ps_mprj_in
#        bit 5~0 - ps_mprj_in[37:32] (Read/Write)
#        others  - reserved
# 0x1c : Data signal of ps_mprj_out
#        bit 31~0 - ps_mprj_out[31:0] (Read)
# 0x20 : Data signal of ps_mprj_out
#        bit 5~0 - ps_mprj_out[37:32] (Read)
#        others  - reserved
# 0x34 : Data signal of ps_mprj_en
#        bit 31~0 - ps_mprj_en[31:0] (Read)
# 0x38 : Data signal of ps_mprj_en
#        bit 5~0 - ps_mprj_en[37:32] (Read)
#        others  - reserved

print ("0x10 = ", hex(ipPS.read(0x10)))
print ("0x14 = ", hex(ipPS.read(0x14)))
print ("0x1c = ", hex(ipPS.read(0x1c)))
print ("0x20 = ", hex(ipPS.read(0x20)))
print ("0x34 = ", hex(ipPS.read(0x34)))
print ("0x38 = ", hex(ipPS.read(0x38)))

In [ ]:
IP_BASE_ADDRESS = 0x60000000
ADDRESS_RANGE = 0x9000
mmio = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)

In [ ]:
# ====================================================================================== #
# ====================================================================================== #
# PL_FSIC Side Configuration
# ====================================================================================== #
# ====================================================================================== #

In [ ]:
# PL_IS Config
ADDRESS_OFFSET = PL_IS #0x7000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

In [ ]:
mmio.write(ADDRESS_OFFSET, 0x12345671)
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

In [ ]:
mmio.write(ADDRESS_OFFSET, 0x12345673)
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

In [ ]:
# PL_DMA Config
ADDRESS_OFFSET = PL_DMA # 0x8000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
print("mmio.read(ADDRESS_OFFSET+0x10): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))
print("mmio.read(ADDRESS_OFFSET+0x20): ", hex(mmio.read(ADDRESS_OFFSET+0x20)))
print("mmio.read(ADDRESS_OFFSET+0x28): ", hex(mmio.read(ADDRESS_OFFSET+0x28)))
print("mmio.read(ADDRESS_OFFSET+0x30): ", hex(mmio.read(ADDRESS_OFFSET+0x30)))
print("mmio.read(ADDRESS_OFFSET+0x38): ", hex(mmio.read(ADDRESS_OFFSET+0x38)))
print("mmio.read(ADDRESS_OFFSET+0x3C): ", hex(mmio.read(ADDRESS_OFFSET+0x3C)))

In [ ]:
# PL_AS Config
ADDRESS_OFFSET = PL_AS # 0x6000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
mmio.write(ADDRESS_OFFSET, 0x12345676)
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

In [ ]:
# PL_AA Config
ADDRESS_OFFSET = PL_AA # 0x2100
print("mmio.read(ADDRESS_OFFSET+0x00): ", hex(mmio.read(ADDRESS_OFFSET+0x00)))
print("mmio.read(ADDRESS_OFFSET+0x04): ", hex(mmio.read(ADDRESS_OFFSET+0x04))) 

In [ ]:
mmio.write(ADDRESS_OFFSET+0x00, 0x11111111)
mmio.write(ADDRESS_OFFSET+0x04, 0x22222222)

In [ ]:
# PL_AA_MB Mailbox
ADDRESS_OFFSET = PL_AA_MB # 0x2000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
print("mmio.read(ADDRESS_OFFSET+0x04): ", hex(mmio.read(ADDRESS_OFFSET+0x04)))
print("mmio.read(ADDRESS_OFFSET+0x08): ", hex(mmio.read(ADDRESS_OFFSET+0x08)))
print("mmio.read(ADDRESS_OFFSET+0x0C): ", hex(mmio.read(ADDRESS_OFFSET+0x0C)))
print("mmio.read(ADDRESS_OFFSET+0x10): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))
print("mmio.read(ADDRESS_OFFSET+0x14): ", hex(mmio.read(ADDRESS_OFFSET+0x14)))
print("mmio.read(ADDRESS_OFFSET+0x18): ", hex(mmio.read(ADDRESS_OFFSET+0x18)))
print("mmio.read(ADDRESS_OFFSET+0x1C): ", hex(mmio.read(ADDRESS_OFFSET+0x1C)))

In [ ]:
mmio.write(ADDRESS_OFFSET, 0x11111112)
mmio.write(ADDRESS_OFFSET+0x04, 0x22222223)
mmio.write(ADDRESS_OFFSET+0x08, 0x33333334)
mmio.write(ADDRESS_OFFSET+0x0C, 0x44444445)
mmio.write(ADDRESS_OFFSET+0x10, 0x55555556)
mmio.write(ADDRESS_OFFSET+0x14, 0x66666667)
mmio.write(ADDRESS_OFFSET+0x18, 0x77777778)
mmio.write(ADDRESS_OFFSET+0x1C, 0x88888889)

In [ ]:
# ====================================================================================== #
# ====================================================================================== #
# PL_Caravel Side Configuration
# ====================================================================================== #
# ====================================================================================== #

In [ ]:
# Caravel-IS Config
ADDRESS_OFFSET = SOC_IS # 0x3000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

In [ ]:
# Caravel-IS Config
ADDRESS_OFFSET = 0x3000
mmio.write(ADDRESS_OFFSET, 0x00000003)

In [ ]:
# Caravel-AS Config
ADDRESS_OFFSET = SOC_AS # 0x4000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

In [ ]:
# Caravel-AS Config
ADDRESS_OFFSET = SOC_AS # 0x4000
mmio.write(ADDRESS_OFFSET, 0x00000006)

In [ ]:
# Caravel-CC Config
ADDRESS_OFFSET = SOC_CC # 0x5000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

In [ ]:
# Caravel-CC Config
ADDRESS_OFFSET = SOC_CC # 0x5000
mmio.write(ADDRESS_OFFSET, 0x00000000)

In [ ]:
# Caravel-LA Config
ADDRESS_OFFSET = SOC_LA # 0x1000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

In [ ]:
# Caravel-UP Config
ADDRESS_OFFSET = SOC_UP # 0x0000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET+0x04)))

In [ ]:
# ====================================================================================== #
# ====================================================================================== #
# LADMA Verification
# ====================================================================================== #
# ====================================================================================== #

In [ ]:
# Allocation memory
ladma_buf = allocate(shape=(1024,), dtype=np.uint32)
print("ladma_buf.device_address: ", hex(ladma_buf.device_address))

IP_BASE_ADDRESS = ladma_buf.device_address
ADDRESS_RANGE = 0x1000
buf_mmio = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
# 0x00 : Control signals
#        bit 0 - ap_start (R/W/COH)
#        bit 1 - ap_done (R/COR)
#        bit 2 - ap_idle (R)
#        bit 3 - ap_ready (R/COR)
# 0x10 : Buffer transfer done status register
#        bit 0 - buffer transfer done status (R)
# 0x20 : Buffer transfer done status clear register
#        bit 0 - clear buffer transfer done status (R/W)
# 0x28 : Buffer Length
#        bit 31~0 - set buffer length (must 1024)
# 0x30 : Triggered condition 
#        bit 23~0 - set triggered condidtion (R/W)
#        others  - reserved
# 0x34 : Buffer Lower base address 
#        bit 31~0 - (R/W)
# 0x38 : Buffer High base address
#        bit 31~0 - (R/W)
# ladma Configuration
ADDRESS_OFFSET = PL_DMA # 0x8000
# exit clear operation
mmio.write(ADDRESS_OFFSET + 0x20, 0x00000000)
# set buffer length
mmio.write(ADDRESS_OFFSET + 0x28, 0x00000400)
# set trigger condition
mmio.write(ADDRESS_OFFSET + 0x30, 0x00000000)
# set buffer low
mmio.write(ADDRESS_OFFSET + 0x38, ladma_buf.device_address)
# set buffer high
mmio.write(ADDRESS_OFFSET + 0x3C, 0x00000000)

In [ ]:
ADDRESS_OFFSET = PL_DMA # 0x8000
print("mmio.read(ADDRESS_OFFSET+0x00): ", hex(mmio.read(ADDRESS_OFFSET+0x00)))
print("mmio.read(ADDRESS_OFFSET+0x10): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))
print("mmio.read(ADDRESS_OFFSET+0x20): ", hex(mmio.read(ADDRESS_OFFSET+0x20)))
print("mmio.read(ADDRESS_OFFSET+0x28): ", hex(mmio.read(ADDRESS_OFFSET+0x28)))
print("mmio.read(ADDRESS_OFFSET+0x30): ", hex(mmio.read(ADDRESS_OFFSET+0x30)))
print("mmio.read(ADDRESS_OFFSET+0x38): ", hex(mmio.read(ADDRESS_OFFSET+0x38)))
print("mmio.read(ADDRESS_OFFSET+0x3C): ", hex(mmio.read(ADDRESS_OFFSET+0x3C)))

In [ ]:
# ladma Configuration
ADDRESS_OFFSET = PL_DMA # 0x8000
# set ap_start
mmio.write(ADDRESS_OFFSET + 0x00, 0x00000001)

In [ ]:
# enable la 0xFFFFFF
ADDRESS_OFFSET = SOC_LA # 0x1000
mmio.write(ADDRESS_OFFSET, 0x00FFFFFF)
#print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
# select target UP
ADDRESS_OFFSET = SOC_CC # 0x5000
mmio.write(ADDRESS_OFFSET, 0x00000003)
#print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

In [ ]:
# ladma Configuration
ADDRESS_OFFSET = PL_DMA # 0x8000
while True:
    if mmio.read(ADDRESS_OFFSET+0x10) == 0x01:
        break
print("mmio.read(ADDRESS_OFFSET+0x10): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))

# disable la 0x000000
ADDRESS_OFFSET = SOC_LA # 0x1000
mmio.write(ADDRESS_OFFSET, 0x00000000)
# select fake UP
ADDRESS_OFFSET = SOC_CC # 0x5000
mmio.write(ADDRESS_OFFSET, 0x00000000) 

ADDRESS_OFFSET = PL_DMA # 0x8000
# clear buffer transfer done operation
mmio.write(ADDRESS_OFFSET + 0x20, 0x00000001)
# set ap_start
mmio.write(ADDRESS_OFFSET + 0x00, 0x00000001)
while True:
    if mmio.read(ADDRESS_OFFSET+0x10) != 0x01:
        break
print("mmio.read(ADDRESS_OFFSET+0x10): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))      

In [ ]:
ADDRESS_OFFSET = PL_DMA # 0x8000
print("mmio.read(ADDRESS_OFFSET+0x00): ", hex(mmio.read(ADDRESS_OFFSET+0x00)))
print("mmio.read(ADDRESS_OFFSET+0x10): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))
print("mmio.read(ADDRESS_OFFSET+0x20): ", hex(mmio.read(ADDRESS_OFFSET+0x20)))
print("mmio.read(ADDRESS_OFFSET+0x28): ", hex(mmio.read(ADDRESS_OFFSET+0x28)))
print("mmio.read(ADDRESS_OFFSET+0x30): ", hex(mmio.read(ADDRESS_OFFSET+0x30)))
print("mmio.read(ADDRESS_OFFSET+0x38): ", hex(mmio.read(ADDRESS_OFFSET+0x38)))
print("mmio.read(ADDRESS_OFFSET+0x3C): ", hex(mmio.read(ADDRESS_OFFSET+0x3C)))

In [ ]:
#dump la log to file
file = open("la_dump.log", "w")
for i in range(0,0xFFF,4):
    file.write('{:08x}'.format(buf_mmio.read(i))+"\n")
file.close()

In [ ]:
# ladma Configuration
ADDRESS_OFFSET = PL_DMA # 0x8000
# exit clear operation
mmio.write(ADDRESS_OFFSET + 0x20, 0x00000000)
# set ap_start again
mmio.write(ADDRESS_OFFSET + 0x00, 0x00000001)

# Confirming SOC_CC & SOC_LA Reading is still work
ADDRESS_OFFSET = SOC_CC # 0x5000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))